# ------------------- Pré‑requis -------------------

```bash
pip install geopandas folium
```

# ------------------- Fusionner les indicateurs -------------------

In [9]:

from pathlib import Path
import pandas as pd, geopandas as gpd

# Répertoire courant = ./notebooks
here = Path.cwd()                      # notebooks/
root = here.parent                     # racine du projet
outputs_dir = here / "outputs"         # notebooks/outputs
data_dir = root / "dataset"            # dataset/

# Chargement des CSV déjà agrégés
price  = pd.read_csv(outputs_dir / "median_price_by_neigh.csv")
occ    = pd.read_csv(outputs_dir / "occupancy_pct_by_neigh.csv")
sent   = pd.read_csv(outputs_dir / "sentiment_by_neigh.csv")

# Chargement du GeoJSON
gdf = gpd.read_file(data_dir / "neighbourhoods.geojson")  # ./dataset/neig.geojson

gdf = (gdf
       .merge(price, on="neighbourhood", how="left")
       .merge(occ,   on="neighbourhood", how="left")
       .merge(sent,  on="neighbourhood", how="left"))

print(gdf.columns)

Index(['neighbourhood', 'neighbourhood_group', 'geometry', 'median_price',
       'avg_occupancy_pct', 'avg_sentiment'],
      dtype='object')


# ------------------- Fonction helper pour créer une choroplèthe Folium -------------------

In [5]:
import folium

def add_choropleth(gdf, column, legend, fmt="{:.0f}"):
    m = folium.Map(location=[45.52, -73.57], zoom_start=11,
                   tiles="CartoDB positron")
    folium.Choropleth(
        geo_data=gdf,
        data=gdf,
        columns=["neighbourhood", column],
        key_on="feature.properties.neighbourhood",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend,
        nan_fill_color="white"
    ).add_to(m)

    # Popup quartier + valeur
    for _, r in gdf.iterrows():
        folium.Popup(f"{r['neighbourhood']}<br>{legend}: "
                     f"{fmt.format(r[column])}").add_to(
            folium.GeoJson(r["geometry"],
                           style_function=lambda *_: {"fillOpacity": 0})
        )
    return m

# ------------------- Générer les 3 cartes -------------------

In [11]:
map_price = add_choropleth(
    gdf, "median_price", "Prix médian (CAD)", "{:.0f}$"
)
map_occ = add_choropleth(
    gdf, "avg_occupancy_pct", "Taux d'occupation %", "{:.1%}"
)
map_sent = add_choropleth(
    gdf, "avg_sentiment", "Sentiment moyen", "{:.2f}"
)

# Sauvegarde HTML
outputs_dir.mkdir(exist_ok=True)
map_price.save(outputs_dir / "map_price.html")
map_occ.save(outputs_dir / "map_occupancy.html")
map_sent.save(outputs_dir / "map_sentiment.html")

# ------------------- Observations Markdown -------------------

In [16]:
from IPython.display import Markdown

best_price = gdf.sort_values("median_price", ascending=False
                             ).iloc[0][["neighbourhood", "median_price"]]
best_occ   = gdf.sort_values("avg_occupancy_pct", ascending=False
                             ).iloc[0][["neighbourhood", "avg_occupancy_pct"]]
best_sent  = gdf.sort_values("avg_sentiment", ascending=False
                             ).iloc[0][["neighbourhood", "avg_sentiment"]]

Markdown(f"""
### Points saillants – Cartes

- **Quartier le plus cher** : *{best_price.neighbourhood}* (médiane {best_price.median_price:.0f}$).  
- **Quartier le plus occupé** : *{best_occ.neighbourhood}* ({best_occ.avg_occupancy_pct:.1%}).  
- **Quartier au meilleur sentiment** : *{best_sent.neighbourhood}* (score {best_sent.avg_sentiment:.2f}).  
""")


### Points saillants – Cartes

- **Quartier le plus cher** : *L'Île-Bizard-Sainte-Geneviève* (médiane 250$).  
- **Quartier le plus occupé** : *Kirkland* (78.0%).  
- **Quartier au meilleur sentiment** : *Kirkland* (score 0.97).  
